In [1]:
from jupyter_dash import JupyterDash
import dash_leaflet as dl
from dash import dcc
from dash import html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output, State
from dash import callback_context
from dash import no_update
import base64
import numpy as np
import pandas as pd
from animalShelter import AnimalShelter

# Removed the external_stylesheets line
app = JupyterDash(__name__)

username = "aacuser"
password = "SNHU1234"
db = AnimalShelter(username, password)
df = pd.DataFrame.from_records(db.read({}))
df.drop(columns=['_id'], inplace=True)

image_filename = 'Grazioso Salvare Logo.png'
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

app.layout = html.Div([
    
    # Logo with URL anchor tag
    html.Center(
    
        html.A(
        
            href='https://www.snhu.edu',        
            target='_blank',
            children=[
            
                html.Img(
                
                    src='data:image/png;base64,{}'.format(encoded_image.decode()),                
                    style={'width': '300px'}
            
                )
        
            ]
    
        )

    ),

    html.Center(
        html.Div(
            [
                html.Div(
                    html.B(html.H4('CS-340 Dashboard - Kyle B. Wucik')),
                    className='card-content'
                )
            ],
            style={'width': '50%', 'border-radius': '15px', 'padding': '10px', 'box-shadow': '0 4px 6px rgba(0,0,0,0.1)', 'background-color': '#f0f0f0'}
        )
    ),
    
    # Filter Dropdown and Reset Button
    html.Hr(),
    html.Div([
    dcc.Dropdown(
    
        id='filter-type',
    
        options=[
            {'label': 'All', 'value': 'All'},
            {'label': 'Water Rescue', 'value': 'Water'},
            {'label': 'Mountain or Wilderness Rescue', 'value': 'Mountain'},
            {'label': 'Disaster or Individual Tracking', 'value': 'Disaster'}
            
        ],
        
        value='All',  # Set the initial value
        multi=False,
        className='col s12 m4 l8'
            
    ),
        
        html.Button('Reset Filters', id='reset-button', n_clicks=0)  # Reset button
    
    ],
      
        className='row'

    
    ),

    html.Hr(),
    html.Div(
        dash_table.DataTable(
            id='datatable-id',
            columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns],
            data=df.to_dict('records'),
            page_size=10,
            sort_action='native',
            row_selectable='single',  # Enable single row selection
            style_table={'overflowX': 'auto', 'box-shadow': '0 4px 6px rgba(0,0,0,0.1)', 'border-radius': '8px'}
        ),
        className='card'
    ),
    
    html.Br(),
    html.Hr(),
    html.Div(
        className='row',
        style={'display': 'flex'},
        children=[
            html.Div(
                id='graph-id',
                className='col s12 m6 card',
                style={'box-shadow': '0 4px 6px rgba(0,0,0,0.1)', 'border-radius': '8px'}  # Add this line
            ),
            html.Div(
                id='map-id',
                className='col s12 m6 card',
                style={'box-shadow': '0 4px 6px rgba(0,0,0,0.1)', 'border-radius': '8px'}  # Add this line
            )
        ])
])

#############################################
# Interaction Between Components / Controller
#############################################


@app.callback (
    
    [Output('datatable-id', 'data'),
    Output('filter-type', 'value')],
    [Input('filter-type', 'value'),
     Input('reset-button', 'n_clicks')],
    prevent_initial_call=True  # Prevents this callback from being triggered on page load

)

def update_dashboard(filter_type, n_clicks):
    
    ctx = callback_context
    triggered_id = ctx.triggered[0]['prop_id'].split('.')[0]

    # If the reset button has been clicked or filter_type is None, return all records and reset the filter to 'All'
    if triggered_id == 'reset-button' or filter_type is None:
        return df.to_dict('records'), 'All'  # Return 'All' for the dropdown value

    # Filtering logic based on rescue types
    if filter_type == 'All':
        return df.to_dict('records'), no_update  # No update to the dropdown value
    elif filter_type == 'Water':
        filtered_df = df[(df['breed'].isin(['Labrador Retriever Mix', 'Chesapeake Bay Retriever', 'Newfoundland'])) & (df['sex_upon_outcome'] == 'Intact Female') & (df['age_upon_outcome_in_weeks'] >= 26) & (df['age_upon_outcome_in_weeks'] <= 156)]
    elif filter_type == 'Mountain':
        filtered_df = df[(df['breed'].isin(['German Shepherd', 'Alaskan Malamute', 'Old English Sheepdog', 'Siberian Husky', 'Rottweiler'])) & (df['sex_upon_outcome'] == 'Intact Male') & (df['age_upon_outcome_in_weeks'] >= 26) & (df['age_upon_outcome_in_weeks'] <= 156)]
    elif filter_type == 'Disaster':
        filtered_df = df[(df['breed'].isin(['Doberman Pinscher', 'German Shepherd', 'Golden Retriever', 'Bloodhound', 'Rottweiler'])) & (df['sex_upon_outcome'] == 'Intact Male') & (df['age_upon_outcome_in_weeks'] >= 20) & (df['age_upon_outcome_in_weeks'] <= 300)]
    else:
        filtered_df = df

    return filtered_df.to_dict('records'), no_update  # No update to the dropdown value


    
@app.callback (
    
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_virtual_data")]
    
)

def update_graphs(viewData):
    
    if viewData is None or len(viewData) == 0:
        return dcc.Graph()  # Return an empty graph if viewData is empty or None

    dff = pd.DataFrame.from_dict(viewData)
    fig = px.pie(dff, names='breed', title='Preferred Animals')
    
    # Update the traces and layout
    fig.update_traces(textposition='inside')
    fig.update_layout(uniformtext_minsize=12, uniformtext_mode='hide')
    
    return dcc.Graph(figure=fig)


    
#This callback will highlight a cell on the data table when the user selects it
@app.callback (

    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]

)

def update_styles(selected_columns):
    if selected_columns is None:
        return []

    return [{
        'if': {'column_id': i},
        'background_color': '#D2F3FF'
    } for i in selected_columns]


# This callback will update the geo-location chart for the selected data entry
# derived_virtual_data will be the set of data available from the datatable in the form of 
# a dictionary.
# derived_virtual_selected_rows will be the selected row(s) in the table in the form of
# a list. For this application, we are only permitting single row selection so there is only
# one value in the list.
# The iloc method allows for a row, column notation to pull data from the datatable

@app.callback (
    
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")]

)

def update_map(viewData, index):  
    if viewData is None:
        #print("viewData is None")
        return
    dff = pd.DataFrame.from_dict(viewData)
    #print("Data Frame:", dff.head())  # Print the first few rows of the DataFrame

    if index is None or len(index) == 0:
        #print("index is None or empty")
        return
    
    row = index[0]
    #print("Selected Row Index:", row)
    #print("Latitude:", dff.iloc[row, 13])
    #print("Longitude:", dff.iloc[row, 14])
        
    # Austin TX is at [30.75,-97.48]
    return [
        
        dl.Map(style={'width': '1000px', 'height': '500px'}, center=[30.75,-97.48], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            dl.Marker(position=[dff.iloc[row,13],dff.iloc[row,14]], children=[
                dl.Tooltip(dff.iloc[row,4]),
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P(dff.iloc[row,9])
                    
                ])
            
            ])
            
        ])
        
    ]


app.run_server(debug=True)

Dash app running on http://127.0.0.1:10685/
